In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import skew

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


from time import time


%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

In [3]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [4]:
train.tail()

In [5]:
targets = train.columns.tolist()[-3:]
feats = train.columns.tolist()[:-3]

feature_data = pd.concat((train[feats],
                          test[feats]))

feature_data.datetime = pd.to_datetime(feature_data.datetime)

feature_data['hour'] = feature_data.datetime.dt.hour
feature_data['day'] = feature_data.datetime.dt.day
feature_data['dayofweek'] = feature_data.datetime.dt.dayofweek
feature_data.drop('datetime', axis=1, inplace=True)

feats = ['hour', 'day', 'dayofweek', 'season', 'holiday', 
         'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']

feature_data = pd.DataFrame(feature_data, columns=feats)

In [8]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
feature_data.hist(bins=40);

In [9]:
feature_data['windspeed'] = np.log1p(feature_data['windspeed'])
X_train = feature_data[:train.shape[0]]
X_test = feature_data[train.shape[0]:]
Y_train = np.log1p(train.loc[:, 'casual':'count'])


In [10]:
Y_train.hist(bins=30);

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from time import time

In [39]:
regs = [GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=180),
        GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=242),
        GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=256)]

d = {}

for reg, target in zip(regs, targets):
    d[target] = np.expm1(reg.fit(X_train, Y_train[target]).predict(X_test))